In [1]:
from collections import deque
from defaultlist import defaultlist

class IntcomputerHaltedException(Exception):
    pass

class Intcomputer:
    def __init__(self, program: list, debug: bool=False, inputs: list=[]):
        self.state = defaultlist(0)
        self.state += program
        self.pc = 0
        self.relbase = 0
        self.debug = debug
        self.input = deque()
        self.output = deque()
        for i in inputs:
            self.input.append(i)
        
    def _op_len(self, op: int) -> int:
        lens = {1: 4, 2: 4, 3: 2, 4: 2, 5: 3, 6: 3, 7: 4, 8: 4, 9: 2, 99: 1}
        return lens[op]
        
    def execute_at(self, addr: int) -> int:
        opcode = self.state[addr] % 100
        modes = list("%03d" % (self.state[addr] / 100))
        modes.reverse()
        oargs = self.state[addr + 1:addr + self._op_len(opcode)]
        args = oargs[:]
        
        if self.debug:
            print(self.state[addr], args, modes, len(self.state), self.relbase)
        
        # Convert positional arguments to immediate values
        # oargs stores original arguments
        for i in range(self._op_len(opcode) - 1):
            if int(modes[i]) == 0: # Positional mode
                args[i] = self.state[args[i]]
            elif int(modes[i]) == 1: # Immediate mode
                pass
            elif int(modes[i]) == 2: # Relative mode
                args[i] = self.state[args[i] + self.relbase]
                oargs[i] += self.relbase
        
        if self.debug:
            print("* %s %s" % (args, oargs))
        
        if opcode == 1: # Add
            self.state[oargs[2]] = args[0] + args[1]
        elif opcode == 2: # Multiply
            self.state[oargs[2]] = args[0] * args[1]
        elif opcode == 3: # Input
            self.state[oargs[0]] = self.input.popleft()
            if self.debug:
                print("Input: %d" % self.state[oargs[0]])
        elif opcode == 4: # Output
            if self.debug:
                print("Output: %d" % self.state[oargs[0]])
            self.output.append(self.state[oargs[0]])
        elif opcode == 5: # Jump-if-true
            if args[0] != 0:
                return args[1]
        elif opcode == 6: # Jump-if-false
            if args[0] == 0:
                return args[1]
        elif opcode == 7: # Less than
            self.state[oargs[2]] = 1 if args[0] < args[1] else 0
        elif opcode == 8: # Equals
            self.state[oargs[2]] = 1 if args[0] == args[1] else 0
        elif opcode == 9: # Adjust relative base
            self.relbase += args[0]
        elif opcode == 99: # Halt
            return None
        else:
            raise Exception("Unknown opcode %d" % opcode)
        return self.pc + self._op_len(opcode)
    
    def step(self) -> bool:
        r = self.execute_at(self.pc)
        if r is None:
            return True
        else:
            self.pc = r
            return False
    
    def run(self):
        while not self.step():
            if self.debug:
                print(" -> %d" % self.pc)
            pass
        
    def run_to_output(self, size=1):
        while len(self.output) < size:
            if self.step():
                raise IntcomputerHaltedException()
    
    def read(self, addr: int) -> int:
        return self.state[addr]
    
    def read_output(self) -> list:
        r = []
        while len(self.output) > 0:
            r.append(self.output.popleft())
        return r
    
    def parse_program_file(fn: str) -> list:
        with open(fn) as f:
            return [int(i) for i in f.readline().strip().split(',')]
                           